# 9 Training a Deep Convolutional Autoencoder (DCA) on MNIST
This example illustrates a DCA architecture in Tensorflow/Keras. It learn on the MNIST dataset to reconstruct 28x28 grayscale images. It uses the **ADAM** optimizer to minimize an **MSE** reconstruction error.

Except the usual convolutional and max pooling layer it has the **Deconvolutional** layers. The code is the activation of the neurons that belong to a fully-connected (dense) layer.

In [ ]:
# Mount GDrive, change directory and check contents of folder.

import os
from google.colab import drive
from google.colab import files

PROJECT_FOLDER = "/content/gdrive/My Drive/Colab Notebooks/CS345_SP22/9. Autoencoders"

drive.mount('/content/gdrive/')
os.chdir(PROJECT_FOLDER)
print("Current dir: ", os.getcwd())

In [ ]:
import os
import csv
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from mllib.utils import RandomSeed
from clang.cindex import callbacks
from mllib.visualization import CPlot
# __________ | Settings | __________
IS_PLOTING_DATA         = False
IS_DEBUGABLE            = True
IS_RETRAINING           = True
RandomSeed(2022)

# Hyperparameters
For each training experiment, we have the encode and the decoder features for the corresponding convolutional and deconvolutional layers. We also have downsampling and upsampling inside the architecture the bottleneck code layer has way less dimensions than its input activation tensor.

In [ ]:
# __________ | Hyperparameters | __________
CONFIG_DA1 = {
                 "ModelName": "MNIST_DA1"
                ,"DA.InputShape": [28,28,1]
                ,"DA.EncoderFeatures": [64,64]
                ,"DA.CodeDimensions" : 32
                ,"DA.Downsampling"   : [True,True]
                ,"DA.DecoderFeatures": [64,64,1]
                ,"DA.DecoderInputResolution": [7,7]
                ,"DA.UpSampling"     :  [True,True]
                ,"DA.HasBatchNormalization": True
                ,"Training.MaxEpoch": 20
                ,"Training.BatchSize": 500
                ,"Training.LearningRate": 1e-3
            }

CONFIG_DA2 = {
                 "ModelName": "MNIST_DA2"
                ,"DA.InputShape": [28,28,1]
                ,"DA.EncoderFeatures": [32,32,32,32]
                ,"DA.CodeDimensions" : 20
                ,"DA.Downsampling"   : [True,False,True,False]
                ,"DA.DecoderFeatures": [32,32,32,32,1]
                ,"DA.DecoderInputResolution": [7,7]
                ,"DA.UpSampling"     :  [False,True,False,True]
                ,"DA.HasBatchNormalization": True
                ,"Training.MaxEpoch": 20
                ,"Training.BatchSize": 500
                ,"Training.LearningRate": 1e-3
            }
                
CONFIG = CONFIG_DA1

# MNIST
This [MNIST dataset](http://yann.lecun.com/exdb/mnist/) dataset, that dates back to 1998, has become a standard toy dataset to understand the image classification task. It contains 70000 grayscale images of 28x28 dimensions for the handwritten digits 0,1,..9. 
It is already splitted into a training set of 60000 images, while the rest 10000 are used to validate the model

# Dataset loading and previewing
We are reusing an existing dataset in Tensorflow format. We load the data and extract them as numpy arrays to view some images, and later use the target class labels for evaluation.

In [ ]:
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

(oTSData, oVSData), oDataSetInfo = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)
  
# Takes one minibatch out of the dataset. Here the size of the minibatch is the total count of samples
for tImages, tLabels in oVSData.batch(oDataSetInfo.splits['test'].num_examples).take(1):
    nImages            = tImages.numpy()
    nTargetClassLabels = tLabels.numpy()  

print("VS image features tensor shape:" , nImages.shape)
print("VS image targets vector shape :", nTargetClassLabels.shape)

if IS_PLOTING_DATA:
    for nIndex, nSample in enumerate(nImages):
      nLabel = nTargetClassLabels[nIndex]
      if (nIndex >= 0 and nIndex <= 20):
           
        if nIndex == 0:
            print("Image sample shape            :", nSample.shape)
        nImage =  nSample.astype(np.uint8) 
        plt.imshow(nImage[:,:,0], cmap="gray") #https://matplotlib.org/stable/tutorials/colors/colormaps.html
        #plt.imshow(nImage[4:22, 0:15, :], cmap="gray") #https://matplotlib.org/stable/tutorials/colors/colormaps.html
        plt.title("Digit %d" % nLabel)
        plt.show()    

# Data Feeding for Training and Validation

In [ ]:
# -----------------------------------------------------------------------------------
def NormalizeImage(p_tImage, p_tLabel):
    # Normalizes color component values from `uint8` to `float32`.
    tNormalizedImage = tf.cast(p_tImage, tf.float32) / 255.
    # Target class labels into one-hot encoding
    tTargetImage = tNormalizedImage 
    
    return tNormalizedImage, tTargetImage
# -----------------------------------------------------------------------------------

nBatchSize = CONFIG["Training.BatchSize"]

# Training data feed pipeline
oTSData = oTSData.map(NormalizeImage, num_parallel_calls=tf.data.AUTOTUNE)
oTSData = oTSData.cache()
oTSData = oTSData.shuffle(oDataSetInfo.splits['train'].num_examples)
oTSData = oTSData.batch(nBatchSize)
oTSData = oTSData.prefetch(tf.data.AUTOTUNE)
print("Training data feed object:", oTSData)

# Validation data feed pipeline
oVSData = oVSData.map(NormalizeImage, num_parallel_calls=tf.data.AUTOTUNE)
oVSData = oVSData.batch(oDataSetInfo.splits['test'].num_examples)
print("Validation data feed object:", oVSData)

# Custom Convolutional Autoencoder Architecture
After several convolutional layers we have a rank-3 activation tensor e.g. 7x7x64. We flatten this tensor to be used as input to our dense code layer. The output of this dense layer is fed input another dense layer that is the first part of the decoder and has 7*7*64 features. We reshape it to a rank-3 tensor that is fed to the first deconvolution operation.


In [ ]:

# __________ // Create the Machine Learning model and training algorithm objects \\ __________
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from mllib.helpers import CKerasModelStructure, CModelConfig

# =========================================================================================================================
class CConvolutionalAutoencoder(keras.Model):
    # --------------------------------------------------------------------------------------
    # Constructor
    def __init__(self, p_oConfig):
        super(CConvolutionalAutoencoder, self).__init__()
        
        # ..................... Object Attributes ...........................
        self.Config = CModelConfig(self, p_oConfig)
        
        
        self.EncoderFeatures    = self.Config.Value["DA.EncoderFeatures"]
        self.DecoderFeatures    = self.Config.Value["DA.DecoderFeatures"]
        self.Downsampling       = self.Config.Value["DA.Downsampling"]
        self.Upsampling         = self.Config.Value["DA.UpSampling"]
        self.Structure = None 
        
        # ......... Keras layers .........
        self.CodeFlatteningLayer = None
        self.CodeDenseLayer      = None
        self.KerasLayers         = []
        # ...................................................................
        
        self.Config.DefaultValue("DA.ActivationFunction", "relu")
        self.Config.DefaultValue("DA.ConvHasBias", False)
        self.Config.DefaultValue("DA.HasBatchNormalization", False)
        self.Config.DefaultValue("DA.KernelInitializer", "glorot_uniform")
        self.Config.DefaultValue("DA.BiasInitializer", "zeros")
        self.Config.DefaultValue("Training.RegularizeL2", False)
        self.Config.DefaultValue("Training.WeightDecay", 1e-5)
            
        if self.Config.Value["Training.RegularizeL2"]:
            print("Using L2 regularization of weights with weight decay %.6f" % self.Config["Training.WeightDecay"])
              
        self.Create()
    # --------------------------------------------------------------------------------------------------------
    def createWeightRegulizer(self):
        if self.Config.Value["Training.RegularizeL2"]:
            oWeightRegularizer = regularizers.L2(self.Config.Value["Training.WeightDecay"])
        else:
            oWeightRegularizer = None
        return oWeightRegularizer          
    # --------------------------------------------------------------------------------------
    def Create(self):
        for nIndex,nFeatures in enumerate(self.EncoderFeatures):
            nStride = 1
            if self.Downsampling[nIndex]:
                nStride = 2
            oConvolution = layers.Conv2D(nFeatures, kernel_size=(3,3), strides=nStride, padding="same"
                                  , use_bias=self.Config.Value["DA.ConvHasBias"]
                                  , kernel_initializer=self.Config.Value["DA.KernelInitializer"]
                                  , bias_initializer=self.Config.Value["DA.BiasInitializer"]
                                  , kernel_regularizer=self.createWeightRegulizer()                            
                                  )
            self.KerasLayers.append(oConvolution)
              
            oActivation  = layers.Activation(self.Config.Value["DA.ActivationFunction"])
            self.KerasLayers.append(oActivation)
            
            if self.Config.Value["DA.HasBatchNormalization"]:
                oNormalization = layers.BatchNormalization()
                self.KerasLayers.append(oNormalization)

        #https://github.com/Seratna/TensorFlow-Convolutional-AutoEncoder
        self.CodeFlatteningLayer = layers.Flatten()
        self.KerasLayers.append(self.CodeFlatteningLayer)
        
        self.CodeDenseLayer      = layers.Dense(self.Config.Value["DA.CodeDimensions"], activation="relu")
        self.KerasLayers.append(self.CodeDenseLayer)
        
        
        nDecoderInputResolution = self.Config.Value["DA.DecoderInputResolution"]
        oDecoderFirstLayer = layers.Dense(nDecoderInputResolution[0]*nDecoderInputResolution[1]*self.DecoderFeatures[0], activation="relu")
        self.KerasLayers.append(oDecoderFirstLayer)
        
        oReshape = layers.Reshape([nDecoderInputResolution[0],nDecoderInputResolution[1],self.DecoderFeatures[0]])
        self.KerasLayers.append(oReshape)
                                
           
        for nIndex,nFeatures in enumerate(self.DecoderFeatures[1:]):
            nStride = 1
            if self.Upsampling[nIndex]:
                nStride = 2
            oDeconvolution = layers.Conv2DTranspose( nFeatures, kernel_size=(3,3), strides=nStride, padding="same"
                                              , use_bias=self.Config.Value["DA.ConvHasBias"]
                                              , kernel_initializer=self.Config.Value["DA.KernelInitializer"]
                                              , bias_initializer=self.Config.Value["DA.BiasInitializer"]
                                              , kernel_regularizer=self.createWeightRegulizer()                            
                                              )
            self.KerasLayers.append(oDeconvolution)
              
            oActivation  = layers.Activation(self.Config.Value["DA.ActivationFunction"])
            self.KerasLayers.append(oActivation)
            
            if self.Config.Value["DA.HasBatchNormalization"]:
                oNormalization = layers.BatchNormalization()
                self.KerasLayers.append(oNormalization)
                
        oLastLayerActivation = layers.Activation("sigmoid")
        self.KerasLayers.append(oLastLayerActivation)
        
    # --------------------------------------------------------------------------------------------------------
    def call(self, p_tInput):
        bPrint = self.Structure is None
        if bPrint:
            self.Structure = CKerasModelStructure()
          
        self.Input = p_tInput
        
        # ....... Convolutional Feature Extraction  .......
        # Feed forward to the next layer
        tA = p_tInput
        if bPrint:
            self.Structure.Add(tA) 
        
        for nIndex,oKerasLayer in enumerate(self.KerasLayers):
            if bPrint:
                self.Structure.Add(tA)         
            tA = oKerasLayer(tA)
        
        return tA
    # --------------------------------------------------------------------------------------------------------
# =========================================================================================================================


# Create the Autoencoder neural network model and training algorithm objects
**Deep Learning techniques**


In [ ]:
oNN = CConvolutionalAutoencoder(CONFIG)

# -----------------------------------------------------------------------------------
def LRSchedule(epoch, lr):
    nNewLR = lr
    for nIndex,oSchedule in enumerate(CONFIG["Training.LearningRateScheduling"]):
        if epoch == oSchedule[0]:
            nNewLR = oSchedule[1]
            print("Schedule #%d: Setting LR to %.5f" % (nIndex+1,nNewLR))
            break
    return nNewLR
# -----------------------------------------------------------------------------------   

nInitialLearningRate    = CONFIG["Training.LearningRate"]  
  
oCostFunction   = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE)
oOptimizer = tf.keras.optimizers.Adam(nInitialLearningRate)
oCallbacks = None

#### Inspect the model architecture

In [ ]:
# Compile the model for training
sModelFolderName = CONFIG["ModelName"]
        
bIsCompiledForTraining = False
if not os.path.isdir(sModelFolderName) or IS_RETRAINING:
    oNN.compile(loss=oCostFunction, optimizer=oOptimizer, metrics=[tf.keras.metrics.RootMeanSquaredError()])
    oNN.predict(oVSData)
    oNN.Structure.Print("Model-Structure-%s.csv" % CONFIG["ModelName"])
    bIsCompiledForTraining = True

### Train and evalute the model

In [ ]:
if bIsCompiledForTraining:
    # Train the model
    if IS_DEBUGABLE:
        oNN.run_eagerly = True
        
    oProcessLog = oNN.fit(  oTSData, batch_size=nBatchSize
                            ,epochs=CONFIG["Training.MaxEpoch"]
                            ,validation_data=oVSData
                            ,callbacks=oCallbacks
                          )
    oNN.summary()          
    oNN.save(sModelFolderName)      
else:
    # The model is trained and its state is saved (all the trainable parameters are saved). We load the model to recall the samples 
    oNN = keras.models.load_model(sModelFolderName)
    oProcessLog = None
    oNN.summary()    


# Learning Process Overview

In [ ]:
if oProcessLog is not None: # [PYTHON] Checks that object reference is not Null
    # list all data in history
    print("Keys of Keras training process log:", oProcessLog.history.keys())
    
    sPrefix = "DA "
            
    # summarize history for accuracy
    sMetricName = "root_mean_squared_error"
    plt.plot(oProcessLog.history[sMetricName])
    plt.plot(oProcessLog.history["val_" + sMetricName])
    plt.title(sPrefix + sMetricName)
    plt.ylabel(sMetricName)
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    
    # summarize history for loss
    
    sCostFunctionNameParts = oCostFunction.name.split("_")                           # [PYTHON]: Splitting string into an array of strings
    sCostFunctionNameParts = [x.capitalize() + " " for x in sCostFunctionNameParts]  # [PYTHON]: List comprehension example 
    sCostFunctionName = " ".join(sCostFunctionNameParts)                             # [PYTHON]: Joining string in a list with the space between them
    
    
    plt.plot(oProcessLog.history['loss'])
    plt.plot(oProcessLog.history['val_loss'])
    plt.title(sPrefix + sCostFunctionName + " Error")
    plt.ylabel('Error')
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

# Inference 
The inference is to feed a set of images and get their reconstructions

In [ ]:
# Takes one minibatch out of the dataset. Here the size of the minibatch is the total count of samples
for tImages, tTargetImages in oVSData.take(1):
    nImages            = tImages.numpy()
    nTargetImages      = tTargetImages.numpy()  

print("VS image features tensor shape:" , nImages.shape)
print("VS image targets tensor shape :" , nTargetImages.shape)

nOutputImages = oNN.predict(nImages)

# Evaluation
We can do **qualitative** evaluation to access the quality of our model in contrast to **quantitative** evaluation that uses metrics.

In [ ]:
for nIndex, nOutputImage in enumerate(nOutputImages[0:10,:]):
    nOriginalImage = nImages[nIndex]*255.0
    nReconstructedImage = nOutputImage*255.0
    print("-"*80)
    plt.imshow(nOriginalImage[:,:,0].astype(np.uint8), cmap="gray") 
    plt.title("Original image samples #%d" % (nIndex+1))
    plt.show()  
    
             
    plt.imshow(nReconstructedImage[:,:,0].astype(np.uint8), cmap="gray") 
    plt.title("Reconstructed image samples #%d" % (nIndex+1))
    plt.show()    